In [19]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import joblib
import warnings
warnings.filterwarnings('ignore')

In [39]:
path = 'C:/Loan/Dataset/train_u6lujuX_CVtuZ9i.csv'

In [41]:
loan_train = pd.read_csv(path)
loan_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [2]:
clf = joblib.load('C:/Loan/decision_tree_model_feature_selection.pkl')

Index(['Credit_History', 'LoanAmount', 'ApplicantIncome', 'CoapplicantIncome','Dependents'],dtype='object')

In [3]:
columns = pd.read_csv('C:/Loan/Dataset/train_u6lujuX_CVtuZ9i.csv', nrows=0).columns[1:-1]
columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area'],
      dtype='object')

In [8]:
columns  = ['Credit_History', 'LoanAmount', 'ApplicantIncome', 'CoapplicantIncome', 'Dependents']

In [50]:
def get_input():
    inputs = []
    for col in columns:
        val = input("Enter {}: ".format(col))
        if col in ['ApplicantIncome', 'CoapplicantIncome']:
            val = int(val)
        elif col in ['LoanAmount', 'Loan_Amount_Term', 'Credit_History']:
            val = float(val)
        inputs.append(val)
    return pd.DataFrame([inputs], columns=columns)

def extract_rules(tree, feature_names):
    left = tree.tree_.children_left
    right = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features = [feature_names[i] for i in tree.tree_.feature]
    # print(left)
    # print(right)
    # print(threshold)
    # print(features)

    def recurse(left, right, threshold, features, node, rules):
        if threshold[node] != -2:
            rules.append((features[node], threshold[node]))
            left_rules = recurse(left, right, threshold, features, left[node], [])
            right_rules = recurse(left, right, threshold, features, right[node], [])
            return rules + left_rules + right_rules
        else:
            return rules
    
    return recurse(left, right, threshold, features, 0, [])
def predict_loan_approval(inputs):
    new_data = inputs[['Credit_History', 'LoanAmount', 'ApplicantIncome', 'CoapplicantIncome', 'Dependents']]
    # new_data.loc[:, 'Dependents'] = new_data['Dependents'].apply(lambda x : {'0' : 0, '1' : 1, '2' : 2, '3+' : 3}[x])
    new_data['Dependents'] = new_data['Dependents'].apply(lambda x : {'0' : 0, '1' : 1, '2' : 2, '3+' : 3}[x])


    y_pred = clf.predict(new_data.values)
    
    print(y_pred)
    
    # Extract the decision path and rules
    decision_path = clf.decision_path(new_data)
    rules = extract_rules(clf, new_data.columns)

    reasons = []
    met_conditions = []
    flags = {'Credit_History': False, 'ApplicantIncome': False, 'LoanAmount': False, 'CoapplicantIncome': False, 'Dependents': False}

    for rule in rules:
        feature, threshold = rule
        value = new_data.iloc[0][feature]

        if feature == 'Credit_History' and float(value) < float(threshold) and not flags[feature]:
            flags[feature] = True
            reasons.append('Customer does not meet credit history requirement.')
        elif feature == 'ApplicantIncome' and float(value) < float(threshold) and not flags[feature]:
            flags[feature] = True
            reasons.append('Customer does not meet income requirement.')
        elif feature == 'LoanAmount' and float(value) > float(threshold) and not flags[feature]:
            flags[feature] = True
            reasons.append('Customer exceeds loan amount requirement.')
        elif feature == 'CoapplicantIncome' and float(value) < float(threshold) and not flags[feature]:
            flags[feature] = True
            reasons.append('Customer does not meet coapplicant income requirement.')
        elif feature == 'Dependents' and float(value) > float(threshold) and not flags[feature]:
            flags[feature] = True
            reasons.append('Customer has too many dependents.')
        elif not flags[feature]:
            flags[feature] = True
            if feature == 'Credit_History':
                met_conditions.append('Customer meets credit history requirement.')
            elif feature == 'ApplicantIncome':
                met_conditions.append('Customer meets income requirement.')
            elif feature == 'LoanAmount':
                met_conditions.append('Customer meets loan amount requirement.')
            elif feature == 'CoapplicantIncome':
                met_conditions.append('Customer meets coapplicant income requirement.')
            elif feature == 'Dependents':
                met_conditions.append('Customer meets dependents requirement.')

    return y_pred[0], reasons, met_conditions


inputs = get_input()
approval_status, reasons, met_conditions = predict_loan_approval(inputs)

if approval_status == 'Y':
    print("Loan application approved.")
    print("Reasons:")
    for condition in met_conditions:
        print(f"- {condition}")
else:
    print("Loan application denied.")
    print("Reasons:")
    for reason in reasons:
        print(f"- {reason}")

['Y']
Loan application approved.
Reasons:
- Customer meets credit history requirement.
- Customer meets loan amount requirement.
- Customer meets income requirement.
